In [114]:
import numpy as np
x = np.random.normal(size=(3, 3, 3))

In [4]:
import numpy as np

class Board2D:
    def __init__(self, **kwargs):
        self.board_size = kwargs.get('board_size', 3)
        self.board = kwargs.get('board', np.zeros(shape=(self.board_size, self.board_size)))
        self.pieces = [-1, 1]
        self.current_player = 0
    
    def __str__(self):
        return str(self.board)
    
    def place(self, loc):
        row, col = loc
        self.board[row, col] = self.pieces[self.current_player]
        self.current_player = (self.current_player + 1)%2
        
    def is_winning_(self, arr, piece):
        return all(arr == piece)
            
    def check_diag(self):
        diag = np.diagonal(self.board)
        off_diag = np.diagonal(np.fliplr(self.board))
        for arr in [diag, off_diag]:
            for piece in self.pieces:
                if self.is_winning_(arr, piece):
                    return piece

        return None
    
    def check_rows(self):
        for row_ix in range(self.board_size):
            row = self.board[row_ix, :]
            for piece in self.pieces:
                if self.is_winning_(row, piece):
                    return piece
        return None

    def check_cols(self):
        for col_ix in range(self.board_size):
            col = self.board[:, col_ix]
            for piece in self.pieces:
                if self.is_winning_(col, piece):
                    return piece
        return None

    def get_winner(self):
        if self.get_legal_moves() is None:
            return 2 # draw
        checks = (self.check_diag(), self.check_rows(), self.check_cols())
        return -1 if -1 in checks else 1 if 1 in checks else 0
    
    def get_legal_moves(self):
        return zip(*np.where(self.board == 0))

In [5]:
class Board3D:
    def __init__(self, **kwargs):
        self.board_size = kwargs.get('board_size', 3)
        self.board = kwargs.get('board', np.zeros(shape=(self.board_size, self.board_size, self.board_size)))
        self.pieces = [-1, 1]
        self.current_player = 0
    
    def __str__(self):
        return str(self.board)
    
    def place(self, loc):
        i, j, k = loc
        self.board[i, j, k] = self.pieces[self.current_player]
        self.current_player = (self.current_player + 1)%2
        
    def is_winning_(self, arr, piece):
        return all(arr == piece)
    
    def get_diags(self):
        out = []
        out.append(np.diagonal(np.einsum('ijj->ij', self.board)))
        out.append(np.diagonal(np.fliplr(np.einsum('ijj->ij', self.board))))
        out.append(np.diagonal(np.einsum('ijj->ij', np.fliplr(self.board))))
        out.append(np.diagonal(np.fliplr(np.einsum('ijj->ij', np.fliplr(self.board)))))
        return out
    
    def get_winner(self):
        if self.get_legal_moves() is None:
            return 2 # draw
        checks = []
        for idx in range(self.board_size):
            checks.append(Board2D(board=self.board[idx, :, :]).get_winner())
            checks.append(Board2D(board=self.board[:, idx, :]).get_winner())
            checks.append(Board2D(board=self.board[:, :, idx]).get_winner())
        
        diags = self.get_diags()
        for diag in diags:
            for piece in self.pieces:
                checks.append(piece if self.is_winning_(diag, piece) else 0)
        
        return -1 if -1 in checks else 1 if 1 in checks else 0
    
    def get_legal_moves(self):
        coords = np.where(self.board == 0)
        return zip(*coords) if len(coords) > 0 else None